In [39]:
# _importing required libraries
import os
import collections

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
# _initializing constant variables
input_file_path = os.getcwd() + f'/../data/processed_data.csv'
subject101_file_path = os.getcwd() + f'/../data/output_csv/subject101_processed.csv'
col_names = ['timestamp (s)', 'activityID', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']

In [45]:
# _loading preprocessed data to main dataframe
main_df = pd.read_csv(input_file_path, header=None, names=col_names)
subject101_df = pd.read_csv(subject101_file_path, header=None, names=col_names)
main_df

/home/praveen/.local/share/virtualenvs/Human_Activity_Recognition-8gA5e_Fl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2
0,37.66,1,2.21530,8.27915,5.58753,-0.004750,0.037579,-0.011145
1,37.67,1,2.29196,7.67288,5.74467,-0.171710,0.025479,-0.009538
2,37.68,1,2.29090,7.14240,5.82342,-0.238241,0.011214,0.000831
3,37.69,1,2.21800,7.14365,5.89930,-0.192912,0.019053,0.013374
4,37.70,1,2.30106,7.25857,6.09259,-0.069961,-0.018328,0.004582
...,...,...,...,...,...,...,...,...
288214,974.50,3,-1.99794,3.94300,9.15686,-0.112651,-0.003501,-0.066523
288215,974.51,3,-2.00276,4.20689,8.96346,-0.179092,0.020300,-0.010637
288216,974.52,3,-1.96042,4.09413,9.11842,-0.285215,0.010714,-0.017346
288217,974.53,3,-1.97860,3.78659,8.65734,-0.347098,0.040476,-0.020536


# Generating subsequences for each sequence of the data

In [25]:
def elbow_techique(sub_sequence_data):
    
    # _only for manual testing
    distortions = []
    for k in range(1,10):
        kmeanModel = KMeans(n_clusters=k)
        kmeanModel.fit(sub_sequence_data)
        distortions.append(kmeanModel.inertia_)

    # _plot the distortions to observe the elbow point from the graph
    plt.figure(figsize=(16,8))
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()

In [50]:
def get_cluster_ids(sub_sequence_data, cluster_cnt):
    
    # _passing random state as an argument to make Kmeans clustering deterministic
    kMeans_model = KMeans(n_clusters = cluster_cnt,random_state=1234).fit(sub_sequence_data)
    return kMeans_model

In [27]:
def get_assigned_words(seq_clusters, cluster_words):
    
    # _assign word to each cluster of the subsequence usnig numpy where function
    assigned_words = np.where(seq_clusters != 0, seq_clusters, cluster_words[0])
    for idx in range(1, len(cluster_words)):
         assigned_words = np.where(seq_clusters != idx, assigned_words, cluster_words[idx])
            
    return assigned_words

In [28]:
def generating_numbers(sequence_names):
    
    # _each word contains common prefix of subsequence and unique cluster alphabet
    alphabet = 'ABCDEFGHIJ'
    words_dict = {}
    
    for seq in sequence_names:
        prefix = seq
        words_dict[seq] = [prefix+'_'+alphabet[i] for i in range(len(alphabet))]
        
    return words_dict

In [42]:
# _initializing variables
window_length = 10
window_overlap = 5

sequence_names = col_names[1:]
num_of_subsequences = len(sequence_names)
sub_sequences = [[] for x in range(num_of_subsequences)]
sub_sequences_101 = [[] for x in range(num_of_subsequences)]

In [43]:
window_index = 0
max_window_index = len(main_df.index)

while window_index <= (max_window_index - window_length):

    activity_sequence = main_df[sequence_names[0]][window_index:window_index+window_length].tolist()

    if len(set(activity_sequence)) == 1:
        sub_sequences[0].append(activity_sequence[0])
        
        for idx in range(1, num_of_subsequences):
            sub_sequences[idx].append(main_df[sequence_names[idx]][window_index:window_index+window_length].tolist())
            
    window_index += window_overlap

# _converting into numpy arrays
np_sequences = np.asarray(sub_sequences[1:])
print(np_sequences.shape)

(6, 57624, 10)


In [46]:
window_index = 0
max_window_index = len(subject101_df.index)

while window_index <= (max_window_index - window_length):

    activity_sequence = subject101_df[sequence_names[0]][window_index:window_index+window_length].tolist()

    if len(set(activity_sequence)) == 1:
        sub_sequences_101[0].append(activity_sequence[0])
        
        for idx in range(1, num_of_subsequences):
            sub_sequences_101[idx].append(subject101_df[sequence_names[idx]][window_index:window_index+window_length].tolist())
            
    window_index += window_overlap

# _converting into numpy arrays
np_sequences_101 = np.asarray(sub_sequences_101[1:])
print(np_sequences_101.shape)

(6, 14460, 10)


# Finding the statistics of the clusters(mean,variance,skewness)


In [47]:
# _document dataframe to store word assignments of each window
doc_df = pd.DataFrame(columns=col_names[1:])
doc_df['activityID'] = sub_sequences_101[0]
doc_df

,activityID,X1,Y1,Z1,X2,Y2,Z2
0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
14455,3,NaN,NaN,NaN,NaN,NaN,NaN
14456,3,NaN,NaN,NaN,NaN,NaN,NaN
14457,3,NaN,NaN,NaN,NaN,NaN,NaN
14458,3,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
words_dict = generating_numbers(sequence_names[1:])
print(words_dict)
print()

# _after observing the clustering result with elbow method, we are manually choosing the n_clusters value.
# _testing: elbow_techique(np_sequences[0])
sequence_cluster_cnts = {'X1': 3, 'Y1': 4, 'Z1': 3, 'X2': 3, 'Y2': 3, 'Z2': 3}
print(sequence_cluster_cnts)

{'X1': ['X1_A', 'X1_B', 'X1_C', 'X1_D', 'X1_E', 'X1_F', 'X1_G', 'X1_H', 'X1_I', 'X1_J'], 'Y1': ['Y1_A', 'Y1_B', 'Y1_C', 'Y1_D', 'Y1_E', 'Y1_F', 'Y1_G', 'Y1_H', 'Y1_I', 'Y1_J'], 'Z1': ['Z1_A', 'Z1_B', 'Z1_C', 'Z1_D', 'Z1_E', 'Z1_F', 'Z1_G', 'Z1_H', 'Z1_I', 'Z1_J'], 'X2': ['X2_A', 'X2_B', 'X2_C', 'X2_D', 'X2_E', 'X2_F', 'X2_G', 'X2_H', 'X2_I', 'X2_J'], 'Y2': ['Y2_A', 'Y2_B', 'Y2_C', 'Y2_D', 'Y2_E', 'Y2_F', 'Y2_G', 'Y2_H', 'Y2_I', 'Y2_J'], 'Z2': ['Z2_A', 'Z2_B', 'Z2_C', 'Z2_D', 'Z2_E', 'Z2_F', 'Z2_G', 'Z2_H', 'Z2_I', 'Z2_J']}

{'X1': 3, 'Y1': 4, 'Z1': 3, 'X2': 3, 'Y2': 3, 'Z2': 3}


In [51]:
for idx, (seq, cluster_cnt) in enumerate(sequence_cluster_cnts.items()):
    
    # _perform k means clustering on subsequences
    KMeans_models = get_cluster_ids(np_sequences[idx], cluster_cnt)
    seq_clusters = KMeans_models.predict(np_sequences_101[idx])
    
    cluster_words = words_dict[seq][:cluster_cnt]
    
    print(f'{idx} -- {seq} -- {cluster_cnt} -- {cluster_words} -- {set(seq_clusters)}')
    
    # _get assigned words and fill the values in doc_df
    assigned_words = get_assigned_words(seq_clusters, cluster_words)
    doc_df[seq] = assigned_words

0 -- X1 -- 3 -- ['X1_A', 'X1_B', 'X1_C'] -- {0, 1, 2}
1 -- Y1 -- 4 -- ['Y1_A', 'Y1_B', 'Y1_C', 'Y1_D'] -- {0, 1, 2, 3}
2 -- Z1 -- 3 -- ['Z1_A', 'Z1_B', 'Z1_C'] -- {0, 1, 2}
3 -- X2 -- 3 -- ['X2_A', 'X2_B', 'X2_C'] -- {0, 1, 2}
4 -- Y2 -- 3 -- ['Y2_A', 'Y2_B', 'Y2_C'] -- {0, 1, 2}
5 -- Z2 -- 3 -- ['Z2_A', 'Z2_B', 'Z2_C'] -- {0, 1, 2}


doc_df

In [52]:
doc_df

,activityID,X1,Y1,Z1,X2,Y2,Z2
0,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
1,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
2,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
3,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
4,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
...,...,...,...,...,...,...,...
14455,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
14456,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
14457,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A
14458,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A


In [35]:
# doc_df.drop_duplicates(keep=False, inplace=True)
# doc_df

In [53]:
# _combine individual words as documents
doc_df['final_sub_sequence'] = doc_df[col_names[2:]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
doc_df

,activityID,X1,Y1,Z1,X2,Y2,Z2,final_sub_sequence
0,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
1,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
2,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
3,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
4,1,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
...,...,...,...,...,...,...,...,...
14455,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
14456,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
14457,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A
14458,3,X1_A,Y1_C,Z1_A,X2_B,Y2_A,Z2_A,X1_A Y1_C Z1_A X2_B Y2_A Z2_A


In [54]:
# _save the combined values to text files
for activity in doc_df['activityID'].unique():
    output_filepath = os.getcwd() + f'/../data/sub_sequence_output/activity_subseq_' + str(activity) + '.txt'
    doc_df.loc[doc_df['activityID'] == activity][['final_sub_sequence']].to_csv(output_filepath, sep='\t', index=False, header= False)